# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](https://github.com/kep1n/lab-neural-networks/blob/master/your-code/tttboard.jpg?raw=1)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [8]:
import pandas as pd
import numpy as np
df = pd.read_csv('../tic-tac-toe.csv')
df.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [6]:
df.isnull().sum()

,0
TL,0
TM,0
TR,0
ML,0
MM,0
MR,0
BL,0
BM,0
BR,0
class,0


In [86]:
# Let's replace
columns = df.columns
for column in columns:
  df[column].replace({"b": 0, "x": 1, "o": 2}, inplace=True)
df['class'] = df['class'].apply(lambda x: 1 if True else 0)
df.head()

/tmp/ipython-input-3846619363.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].replace({"b": 0, "x": 1, "o": 2}, inplace=True)


,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,1,1,1,1,2,2,1,2,2,1
1,1,1,1,1,2,2,2,1,2,1
2,1,1,1,1,2,2,2,2,1,1
3,1,1,1,1,2,2,2,0,0,1
4,1,1,1,1,2,2,0,2,0,1


In [92]:
features = df[columns[:-1]]
target = df[columns[-1]]


In [93]:
predictors_norm = (features - features.mean()) / features.std()
predictors_norm.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR
0,-0.183675,-0.10789,-0.183675,-0.10789,1.163573,1.184095,-0.183675,1.184095,1.169866
1,-0.183675,-0.10789,-0.183675,-0.10789,1.163573,1.184095,1.169866,-0.107890,1.169866
2,-0.183675,-0.10789,-0.183675,-0.10789,1.163573,1.184095,1.169866,1.184095,-0.183675
3,-0.183675,-0.10789,-0.183675,-0.10789,1.163573,1.184095,1.169866,-1.399876,-1.537216
4,-0.183675,-0.10789,-0.183675,-0.10789,1.163573,1.184095,-1.537216,1.184095,-1.537216


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [94]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

In [95]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

In [96]:
n_classes = len(np.unique(y_train))
n_classes

1

In [98]:
def sequential_model():
    global n_classes
    seq_model = Sequential()
    seq_model.add(Dense(30, activation='relu', input_shape=(len(columns[:-1]),)))
    seq_model.add(Dense(30, activation='relu'))
    seq_model.add(Dense(30, activation='relu'))
    seq_model.add(Dense(2, activation='softmax'))

    seq_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return seq_model

seq_model = sequential_model()
seq_model.fit(X_train, y_train, epochs=100, verbose=2)
y_pred = seq_model.predict(X_test)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
21/21 - 2s - 113ms/step - accuracy: 0.7179 - loss: 0.6031
Epoch 2/100
21/21 - 0s - 4ms/step - accuracy: 1.0000 - loss: 0.2659
Epoch 3/100
21/21 - 0s - 4ms/step - accuracy: 1.0000 - loss: 0.0657
Epoch 4/100
21/21 - 0s - 4ms/step - accuracy: 1.0000 - loss: 0.0146
Epoch 5/100
21/21 - 0s - 5ms/step - accuracy: 1.0000 - loss: 0.0057
Epoch 6/100
21/21 - 0s - 5ms/step - accuracy: 1.0000 - loss: 0.0032
Epoch 7/100
21/21 - 0s - 4ms/step - accuracy: 1.0000 - loss: 0.0020
Epoch 8/100
21/21 - 0s - 4ms/step - accuracy: 1.0000 - loss: 0.0014
Epoch 9/100
21/21 - 0s - 4ms/step - accuracy: 1.0000 - loss: 0.0011
Epoch 10/100
21/21 - 0s - 4ms/step - accuracy: 1.0000 - loss: 8.0896e-04
Epoch 11/100
21/21 - 0s - 5ms/step - accuracy: 1.0000 - loss: 6.3485e-04
Epoch 12/100
21/21 - 0s - 4ms/step - accuracy: 1.0000 - loss: 5.0884e-04
Epoch 13/100
21/21 - 0s - 4ms/step - accuracy: 1.0000 - loss: 4.1406e-04
Epoch 14/100
21/21 - 0s - 5ms/step - accuracy: 1.0000 - loss: 3.4434e-04
Epoch 15/100
21/21 - 

In [99]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred_classes = np.argmax(y_pred, axis=1)
print("Accuracy:", accuracy_score(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes))
print(confusion_matrix(y_test, y_pred_classes))

Accuracy: 1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       288

    accuracy                           1.00       288
   macro avg       1.00      1.00      1.00       288
weighted avg       1.00      1.00      1.00       288

[[288]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [102]:
seq_model.save('../tic-tac-toe.keras')

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [44]:
loaded_model = keras.models.load_model('../tic-tac-toe.keras')

In [80]:
row = X_test.iloc[[16]]
row

,TL,TM,TR,ML,MM,MR,BL,BM,BR
30,-0.183675,-0.10789,-0.183675,1.184095,1.163573,-1.399876,1.169866,-1.399876,-0.183675


In [81]:
prediction = loaded_model.predict(row)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


In [82]:
y_pred_classes = np.argmax(prediction, axis=1)

In [83]:
y_pred_classes

array([1])

In [84]:
y_test.iloc[16]

np.True_

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here